# ブロックとProc

* ブロック
    * 新たにスコープを作成する
        * for式、if式、while式などはスコープは作成されない
    * <font color="red">メソッドを呼び出すときのみ記述できる</font>
    * yieldという式を使う事でブロック内部で記述した式を呼び出すことができる
    * 「{}」と「do ～ end」の形式で書ける
        * 傾向
            * 1行で書けるときは{}
            * 複数行で書く時は do end
            
* 参考
    * [メタプログラミングRuby 第3章 まとめ　(ブロック)](http://portaltan.hatenablog.com/entry/2015/07/17/181358)

## yieldで実行結果の取得

* 「{}」に囲まれているのがブロック
    * ここのブロックは2を返す
    * メソッドfunc
        * ブロックの実行結果(yield)と引数の合計3を返している

In [13]:
def func x
  x + yield
end
puts func(1){ 2 }

### do ～ end の形式でも書ける
# func(1) do
#   2
# end

3


## スコープの生成

* ブロックの中で変数に代入された値はブロックの外では参照できない

In [27]:
def func x
  x + yield
end
func(1){ z = 2 }

### ブロック内で作成された変数zはブロックの外では参照できない
puts z

NameError: undefined local variable or method `z' for main:Object

## クロージャとしてのブロック

* クロージャ
    * 処理の生成時の環境を束縛するもの
* おそらく
    * クロージャはブロック(メソッド)の外で作った変数を利用しているブロックのこと
    * 束縛はクロージャで使う変数をそれっぽく難しく言ったもの
* 挙動
    * ブロックの外で変数yに2を代入 (束縛yと表現する？)
    * メソッドfuncにブロックを渡し、yieldによりブロックを実行
        * このときyの値を取得して更新する
        * このyは、ブロックの外と同じy
            * <font color="red">値ではなく変数そのものが共有されている</font>
* Rubyはメソッド内部から外部の変数を参照できない
    * クロージャが呼び出し元の変数を処理に持ち込む数少ない手段の一つ

In [24]:
def func x
  x + yield
end

### ブロックの外で変数yに値を代入
y = 2

### このブロック内のyは先ほどのブロックの外の変数yと同一
puts func(1){y+=2}

### yの値は更新されている
puts y

5
4


## ブロックの引数の指定

* funcに1と2を渡す
* func内部で以下合計する
    * 第一引数の値1
    * ブロックの実行結果
        * yieldはブロック引数2,3を合計して5を返す
* 6が返る

In [28]:
def func a, b
  a + yield(b, 3)
end

puts func(1, 2){|x,y| x+y}

6


## ブロックの判定(ブロックが指定されたかどうか)

* block_given?メソッドで実現できる

In [29]:
def func
  return 1 if block_given?
  2
end

puts func(){}
puts func

1
2


## Procの基本

* ブロックをオブジェクトとして渡したい時に利用する
    * ブロックそのものはオブジェクトではない
* 生成
    * Procクラスのコンクラスタにブロックを指定すると生成できる
* 利用
    * Procのインスタンスに対してcallメソッドを使う


In [30]:
proc = Proc.new{|x| puts x}
proc.call(1)

1


## Procオブジェクトの生成

* 遅延評価
    * 処理自体を先に生成して後で評価する
    * 例
        * カウンタで初期値をプログラム冒頭で決定 : 処理と初期値の設定
        * 後の処理でカウントする : 遅延評価

In [44]:
def get_counter start
  Proc.new do |up|
    puts "start= #{start}"
    puts "up= #{up}"
    start += up
  end
end

### 初期値として3を設定。count_upはProcオブジェクトを参照
count_up = get_counter(3)

### 初期値に1を足す
puts count_up.call(1)

### さらに3を足す
puts count_up.call(3)

start= 3
up= 1
4
start= 4
up= 3
7


## Proc ⇔ ブロック 変換

* Proc -> ブロック
    * Procオブジェクトに「&」をつけて最後の引数に指定する
* ブロック -> Proc
    * 最後の仮引数に「&」をつけた引数で参照できる

In [55]:
### Proc -> ブロック
def func x
  x + yield
end
proc = Proc.new{2}

### 最後の引数に&をつけて渡す
puts func(1,&proc)
  
#=========================#
  
### ブロック -> Proc
### 関数の最後の仮引数に&をつけた名前を渡す
def func2 y, &proc2
  y + proc2.call
end
puts func2(3){ 4 }

3
7


## lambda


* lambdaメソッド
    * Procインスタンスを生成する
        * Procインスタンスはlambdaとも呼ばれる
        * Procやブロックよりもメソッドに近い動きをする
    * Proc.newで生成したインスタンはprocとも呼ばれる

In [43]:
lmd = lambda{|x| puts x}
lmd.call(1)

### Ruby1.9以降の新しい書き方 (アロー演算子)
### putsを外出し
lmd2 = -> ( x ) { x }
puts lmd2.call(2)

### putsをブロック内に記載
lmd3 = -> (x, y){puts x + y}
lmd3.call(3, 4)

1
2
7


## Procとlambdaの振る舞いの違い

* returnとスコープ
    * Procは生成元のスコープを脱出する
    * lambdaはブロック内でreturnすると呼び出し元に復帰する

In [25]:
puts "Procはreturnでスコープを脱出するので2まで行かない"
def func_proc
  proc = Proc.new{return 1}
  puts proc.call
  puts 2
 end
puts func_proc

puts "lambdaは呼び出し元に戻るので4まで行く"
def func_lambda
  proc = lambda{return 3}
  puts proc.call
  puts 4
end
func_lambda

Procはcallでスコープを脱出するので2まで行かない
1
lambdaは呼び出し元に戻る
3
4


* 引数が一致しない場合
    * Procは無視、もしくはnilを返す
    * lambdaは例外ArgumentErrorが発生する

In [39]:
puts "引数が一致しない場合はnil"
p1 = Proc.new{|x,y| y}
p p1.call(1,2)
p p1.call(1)

puts "引数が一致しない場合は例外が発生"
l1 = lambda{|x,y| y}
p l1.call(3,4)
p l1.call(3)

引数が一致しない場合はnil
2
nil
引数が一致しない場合は例外が発生
4


ArgumentError: wrong number of arguments (given 1, expected 2)